In [60]:
import pandas as pd
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [61]:
def selectkbest(indep_x,dep_y,n):
    test=SelectKBest(score_func=chi2,k=n)
    fit1=test.fit(indep_x,dep_y)
    selectk_features=fit1.transform(indep_x)
    return selectk_features
    
def split_scalar(indep_x,dep_y):
    x_train,x_test,y_train,y_test=train_test_split(indep_x,dep_y,test_size=0.25,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def R2_prediction(regressor,x_test):
    y_pred=regressor.predict(x_test)
#Making confusion matrix
    from sklearn.metrics import r2_score
    r_score=r2_score(y_test,y_pred)
    from sklearn.metrics import accuracy_score
    accuracy=accuracy_score(y_test,y_pred)
    return regressor,accuracy,x_test,y_test,r_score

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    regressor=LogisticRegression(random_state=0)
    regressor.fit(x_train,y_train)
    regressor,accuracy,x_test,y_test,r_score=R2_prediction(regressor,x_test)
    return regressor,accuracy,x_test,y_test,r_score

def svm_linear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    regressor=SVC(kernel="linear",random_state=0)
    regressor.fit(x_train,y_train)
    regressor,accuracy,x_test,y_test,r_score=R2_prediction(regressor,x_test)
    return regressor,accuracy,x_test,y_test,r_score

def svm_NL(x_train,y_train,x_test):
    from sklearn.svm import SVC
    regressor=SVC(kernel="rbf",random_state=0)
    regressor.fit(x_train,y_train)
    regressor,accuracy,x_test,y_test,r_score=R2_prediction(regressor,x_test)
    return regressor,accuracy,x_test,y_test,r_score

def DecisionTree(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    regressor=DecisionTreeClassifier(criterion="entropy",random_state=0)
    regressor.fit(x_train,y_train)
    regressor,accuracy,x_test,y_test,r_score=R2_prediction(regressor,x_test)
    return regressor,accuracy,x_test,y_test,r_score

def RandomForest(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    regressor=RandomForestClassifier(n_estimators=10, criterion="entropy",random_state=0)
    regressor.fit(x_train,y_train)
    regressor,accuracy,x_test,y_test,r_score=R2_prediction(regressor,x_test)
    return regressor,accuracy,x_test,y_test,r_score

In [62]:
def selectK_Regression(acclog,accsvm1,accsvmn1,accdes,accrf):
    dataframe=pd.DataFrame(index=["chiSquare"],columns=["Logistic","SVM1","SVMn1","Decision","Random"])
    for number,idex in enumerate(dataframe.index):
        dataframe["Logistic"][idex]=acclog[number]
        dataframe["SVM1"][idex]=accsvm1[number]
        dataframe["SVMn1"][idex]=accsvmn1[number]
        dataframe["Decision"][idex]=accdes[number]
        dataframe["Random"][idex]=accrf[number]
    return dataframe    

In [63]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df=pd.get_dummies(df2,drop_first=True)
indep_x=df.drop("classification_yes",axis=1)
dep_y=df["classification_yes"]

In [64]:
df2

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [65]:
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [99]:
kbest=selectkbest(indep_x,dep_y,8)
acclog=[]
accsvm1=[]
accsvmn1=[]
accdes=[]
accrf=[]

In [100]:
kbest

array([[2.00000000e+00, 3.00000000e+00, 1.48112676e+02, ...,
        1.25181556e+01, 3.88689024e+01, 8.40819113e+03],
       [3.00000000e+00, 2.00000000e+00, 1.48112676e+02, ...,
        1.07000000e+01, 3.40000000e+01, 1.23000000e+04],
       [4.00000000e+00, 1.00000000e+00, 9.90000000e+01, ...,
        1.20000000e+01, 3.40000000e+01, 8.40819113e+03],
       ...,
       [5.14923077e+01, 3.00000000e+00, 1.10000000e+02, ...,
        9.10000000e+00, 2.60000000e+01, 9.20000000e+03],
       [5.14923077e+01, 0.00000000e+00, 2.07000000e+02, ...,
        8.50000000e+00, 3.88689024e+01, 8.40819113e+03],
       [5.14923077e+01, 0.00000000e+00, 1.00000000e+02, ...,
        1.63000000e+01, 5.30000000e+01, 8.50000000e+03]])

In [101]:
x_train,x_test,y_train,y_test=split_scalar(kbest,dep_y)

regressor,accuracy,x_test,y_test,r_score=logistic(x_train,y_train,x_test)
acclog.append(accuracy)

regressor,accuracy,x_test,y_test,r_score=svm_linear(x_train,y_train,x_test)
accsvm1.append(accuracy)

regressor,accuracy,x_test,y_test,r_score=svm_NL(x_train,y_train,x_test)
accsvmn1.append(accuracy)

regressor,accuracy,x_test,y_test,r_score=DecisionTree(x_train,y_train,x_test)
accdes.append(accuracy)

regressor,accuracy,x_test,y_test,r_score=RandomForest(x_train,y_train,x_test)
accrf.append(accuracy)

result=selectK_Regression(acclog,accsvm1,accsvmn1,accdes,accrf)


C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [71]:
result
#2

,Logistic,SVM1,SVMn1,Decision,Random
chiSquare,0.75,0.69,0.73,0.72,0.71


In [75]:
result
#3

,Logistic,SVM1,SVMn1,Decision,Random
chiSquare,0.82,0.82,0.82,0.84,0.83


In [78]:
result
#4

,Logistic,SVM1,SVMn1,Decision,Random
chiSquare,0.85,0.82,0.83,0.89,0.89


In [82]:
result
#5

,Logistic,SVM1,SVMn1,Decision,Random
chiSquare,0.94,0.94,0.95,0.96,0.95


In [94]:
result
#6

,Logistic,SVM1,SVMn1,Decision,Random
chiSquare,0.96,0.96,0.96,0.97,0.97


In [98]:
result
#7

,Logistic,SVM1,SVMn1,Decision,Random
chiSquare,0.97,0.97,0.97,0.98,0.97
